In [1]:
import warnings
warnings.filterwarnings('ignore')

# Hyper Parameter

In [2]:
lmbd = 0.1
embed_size = 150
dropout_rate = 0.5
K = 3
epochs = 1
number_of_item = 150

# Data

In [3]:
import pandas as pd
import hypernetx as hp
import re

from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec,KeyedVectors

class Data ():
    def __init__(self):      
        self.all_hyper_edges = []
        self.all_hyper_graphs = []
        self.users = []
        self.data = []        
        
        self.number_of_items = 0 #nodes
        self.number_of_users = 0 #edges
        self.number_of_interactions = 0
        self.static_item_embedding = []
        # self.static_item_embedding = torch.Tensor(self.number_of_items,embed_size)

    def set_data(self , features_files_name , interactions_files_name):
        self.features_dataframe = pd.read_csv(features_files_name)
        self.interactions_dataframe = pd.read_csv(interactions_files_name)
        self.dataset_by_year = dict(tuple(self.interactions_dataframe.groupby("reviewTime")))
        self.item_purched_list , self.year_purched_list , self.user_item_year_list = self.set_purches_list()

    def clear_every_thing(self):
        self.all_hyper_edges = []
        self.all_hyper_graphs = []
        self.users = []
        self.data = []        
        
        self.number_of_items = 0 #nodes
        self.number_of_users = 0 #edges
        self.number_of_interactions = 0
        self.static_item_embedding = []



    def show_dataset_by_year(self):
        for val in self.dataset_by_year:
            print(val)
            print(self.dataset_by_year[val])
            print('\n')

    def get_dataset_by_year(self):
        return self.dataset_by_year

    def get_years(self):
        return self.dataset_by_year.keys()

    def get_user(self):
        self.users = list(set(self.interactions_dataframe['reviewerID']))
        return self.users

    def set_purches_list(self):
        purches_list = defaultdict(list)
        user_year_list = defaultdict(list) # list salhaye kharid karbar 
        user_item_year_list = defaultdict(list)
        for year in range(0,len(self.get_dataset_by_year())):
            df_aYear = list(self.get_dataset_by_year().values())[year]
            plist = []
            for interaction_row in df_aYear.iterrows():
                item_id = interaction_row[1]['asin']
                user_id = interaction_row[1]['reviewerID']
                u_year = interaction_row[1]['reviewTime']
                purches_list[user_id].append(item_id)
                user_year_list[user_id].append(u_year)
                interaction = (item_id,u_year)
                plist.append(interaction)
            user_item_year_list[user_id].append(plist)
            plist = []

        return purches_list , user_year_list , user_item_year_list

    def get_user_purches_item(self,user_id):
        return self.item_purched_list[user_id]

    def get_user_purches_year(self,user_id):
        return self.year_purched_list[user_id]

    def get_user_item_year_list(self):
        return self.user_item_year_list

    def get_all_hyper_edges(self):
        hyper_edge_for_one_hypergraph = {}
        edge_number = []
        for one_year in self.dataset_by_year:
            hyper_edge_for_one_hypergraph = {}
            edge_number = []
            for ind in self.dataset_by_year[one_year].index:
                edge_number.append(self.dataset_by_year[one_year]['reviewerID'][ind])
            edge_number = list(dict.fromkeys(edge_number))
            nodes = []

            for edge in edge_number:
                for ind in self.dataset_by_year[one_year].index:        
                    if edge==int(self.dataset_by_year[one_year]['reviewerID'][ind]):            
                        nodes.append(int(self.dataset_by_year[one_year]['asin'][ind]))
                    elif edge > int(self.dataset_by_year[one_year]['reviewerID'][ind]): 
                        continue
                    else:
                        hyper_edge_for_one_hypergraph[edge]=nodes
                        nodes = []
                        break
            hyper_edge_for_one_hypergraph[edge]=nodes
            self.all_hyper_edges.append(hyper_edge_for_one_hypergraph)
        return self.all_hyper_edges

    def get_all_hyper_graphs(self):
        for i in range(len(self.all_hyper_edges)):
            G = hp.Hypergraph(self.all_hyper_edges[i].values())
            self.all_hyper_graphs.append(G)
        return self.all_hyper_graphs

    def removeNonAscii(self,text):
        return "".join(i for i in text if  ord(i)<128)
    
    def make_lower_case(self,text):
        return text.lower() 

    def remove_html(self,text):
        html_pattern = re.compile('<.*?>')
        return html_pattern.sub(r'', text)

    def remove_stop_words(self,text):
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
        return text   

    def remove_punctuation(self,text):
        tokenizer = RegexpTokenizer(r'\w+')
        text = tokenizer.tokenize(text)
        text = " ".join(text)  
        return text
    
    def remove_punctuation_from_list(self,list):
        string = ""
        for w in list:
            string = " ".join(w)
        return self.remove_punctuation(string)

    def cleaning_data(self):
        # description
        self.features_dataframe = self.features_dataframe[self.features_dataframe['description'].notnull()]
        self.features_dataframe['Cleaned_description'] = self.features_dataframe['description'].apply(self.removeNonAscii)
        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.remove_html)
        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.make_lower_case)
        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.remove_stop_words)
        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.remove_punctuation)

        # categories
        self.features_dataframe = self.features_dataframe[self.features_dataframe['categories'].notnull()]
        self.features_dataframe['Cleaned_categories'] = self.features_dataframe['categories'].apply(self.remove_punctuation_from_list)
        self.features_dataframe['Cleaned_categories'] = self.features_dataframe.Cleaned_categories.apply(func = self.make_lower_case)

        # title
        self.features_dataframe = self.features_dataframe[self.features_dataframe['title'].notnull()]
        self.features_dataframe['Cleaned_title'] = self.features_dataframe['title'].apply(self.removeNonAscii)
        self.features_dataframe['Cleaned_title'] = self.features_dataframe.Cleaned_title.apply(func = self.make_lower_case)
        self.features_dataframe['Cleaned_title'] = self.features_dataframe.Cleaned_title.apply(func = self.remove_stop_words)
        self.features_dataframe['Cleaned_title'] = self.features_dataframe.Cleaned_title.apply(func = self.remove_punctuation)

        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.make_lower_case)
        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.remove_stop_words)
        self.features_dataframe['Cleaned_description'] = self.features_dataframe.Cleaned_description.apply(func = self.remove_punctuation)

        # categories
        self.features_dataframe = self.features_dataframe[self.features_dataframe['categories'].notnull()]
        self.features_dataframe['Cleaned_categories'] = self.features_dataframe['categories'].apply(self.remove_punctuation_from_list)
        self.features_dataframe['Cleaned_categories'] = self.features_dataframe.Cleaned_categories.apply(func = self.make_lower_case)

        # title
        self.features_dataframe = self.features_dataframe[self.features_dataframe['title'].notnull()]
        self.features_dataframe['Cleaned_title'] = self.features_dataframe['title'].apply(self.removeNonAscii)
        self.features_dataframe['Cleaned_title'] = self.features_dataframe.Cleaned_title.apply(func = self.make_lower_case)
        self.features_dataframe['Cleaned_title'] = self.features_dataframe.Cleaned_title.apply(func = self.remove_stop_words)
        self.features_dataframe['Cleaned_title'] = self.features_dataframe.Cleaned_title.apply(func = self.remove_punctuation)

        description = []
        for words in self.features_dataframe['Cleaned_description']:
            description.append(words.split())

        categories = []
        for words in self.features_dataframe['Cleaned_categories']:
            categories.append(words.split())

        title = []
        for words in self.features_dataframe['Cleaned_title']:
            title.append(words.split())

        self.data = list(zip(title,categories,description))
        for i in range(len(self.data)):
            self.data[i] = self.data[i][0] + self.data[i][1] + self.data[i][2]

    def training_google_model(self):
        self.cleaning_data()
        EMBEDDING_FILE = 'E:/google news vector/GoogleNews-vectors-negative300.bin.gz'
        google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary = True)

        # Training our corpus with the model
        google_model = Word2Vec(size = 300, window = 5, min_count = 2, workers = -1)

        google_model.build_vocab(self.data)

        google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf = 1.0, binary = True)

        google_model.train(self.data, total_examples = google_model.corpus_count, epochs = 5)

        return google_model

    def create_static_embedding(self):
        google = self.training_google_model()
        static_item_embeddings = []
        for line in self.data:
            avgword2vec = None
            count = 0
            for word in line:
                if word in google.wv.vocab:
                    count += 1
                    if avgword2vec is None:
                        avgword2vec = google[word]
                    else:
                        avgword2vec = avgword2vec + google[word]

            if avgword2vec is not None:
                avgword2vec = avgword2vec / count
                static_item_embeddings.append(avgword2vec)

        return static_item_embeddings

    def set_static_embedding(self):
        self.static_item_embedding = self.create_static_embedding()
        
        # self.static_item_embedding = static_item_embeddings

    def get_static_embedding(self,index):
        return self.static_item_embedding[index]

    def get_statistic(self):
        self.number_of_items = len(pd.unique(self.interactions_dataframe['asin'])) #nodes
        self.number_of_users = len(pd.unique(self.interactions_dataframe['reviewerID'])) #edges
        self.number_of_interactions = self.interactions_dataframe.shape[0]

        return self.number_of_items , self.number_of_users , self.number_of_interactions
                    

KeyboardInterrupt: 

# Layers

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import hypernetx as hp


from torch.nn import Parameter , TransformerEncoder, TransformerEncoderLayer


class ResidualGAtingLayer(nn.Module):
    def __init__(self,num_item,embed_size):
        super(ResidualGAtingLayer, self).__init__()
        self.W_R = Parameter(data = torch.FloatTensor(embed_size, embed_size), requires_grad=True)
        torch.nn.init.uniform_(self.W_R,-1,1)
        self.Z_R = Parameter(data = torch.FloatTensor(embed_size), requires_grad=True)
        torch.nn.init.uniform_(self.Z_R,-1,1)
        self.G = 0
        self.x = torch.empty((num_item,embed_size))
        self.e = torch.empty((num_item,embed_size))
    
    def setter(self,dynamic_embeddings,static_item_embeddings):
        self.x = dynamic_embeddings
        self.e = static_item_embeddings

    def forward(self):
        self.G = torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.x))) / (torch.exp(torch.transpose(self.Z_R, 0, 0) * torch.tanh(torch.matmul(self.W_R,self.x))) + torch.exp(torch.transpose(self.Z_R, 0, 0) * torch.tanh(self.W_R*self.e)))
        return self.G

    def calculate_residualGAting(self):
        return self.G*self.x + (1-self.G)*self.e


class ShortTermUserIntent(nn.Module):
    def __init__(self,num_item , embed_size ):
        super(ShortTermUserIntent, self).__init__()
        self.W = Parameter(data = torch.Tensor(num_item, embed_size), requires_grad=True)
        torch.nn.init.uniform_(self.W,-1,1)

        self.item_num = num_item
        self.hyperGraph = hp.Hypergraph
        self.incident_matrix = np.empty(embed_size)
        self.incident_matrix_tensor = torch.empty((num_item,embed_size))
        self.H_transpose = torch.empty((num_item,embed_size))       
        self.diagonal_Dv_invert = torch.empty((num_item,embed_size))  
        self.diagonal_De_invert = torch.empty((num_item,embed_size))  

    def setter(self,G):
        self.hyperGraph = G
        self.incident_matrix = np.matrix(self.hyperGraph.incidence_matrix().toarray())
        self.incident_matrix_tensor = torch.tensor(self.hyperGraph.incidence_matrix().toarray()).float()
        self.H_transpose = torch.transpose(self.incident_matrix_tensor, 0, 1)
        Dv = np.array(np.sum(self.incident_matrix, axis=0)).flatten()
        De = np.array(np.sum(self.incident_matrix, axis=1)).flatten()
        invertedDv = []
        for i in Dv:
            if i==0:
                invertedDv.append(i)
            else:
                invertedDv.append(1/np.power(i , 0.5))
        Dv_invert = torch.tensor(invertedDv).float()
        
        self.diagonal_Dv_invert = torch.diag(Dv_invert)

        invertedDe = []
        for i in De:
            if i==0:
                invertedDe.append(i)
            else:
                invertedDe.append(1/np.power(i , 0.5))
        De_invert = torch.tensor(invertedDe).float()
        self.diagonal_De_invert = torch.diag(De_invert)

    def forward(self,X , batch_num):
        temp = torch.matmul(self.H_transpose,self.diagonal_De_invert)
        temp = torch.matmul(self.diagonal_Dv_invert,temp)

        right_index = self.item_num*batch_num 

        edg = []
        for i in list(self.hyperGraph):
            edg.append(i-right_index)
            

        # temp = torch.matmul(temp,X[list(self.hyperGraph)])
        temp = torch.matmul(temp,X[edg])
        temp = torch.matmul(temp,self.W)
        return F.relu(temp)


class FusionLayer(nn.Module):
    def __init__(self,num_item ,embed_size):
        super(FusionLayer, self).__init__()
        self.W_R = Parameter(data = torch.FloatTensor(embed_size, embed_size), requires_grad=True)
        torch.nn.init.uniform_(self.W_R,-1,1)
        self.Z_R = Parameter(data = torch.FloatTensor(embed_size), requires_grad=True)
        torch.nn.init.uniform_(self.Z_R,-1,1)
        self.a_u = 0
        self.a_d = 0
        self.x = torch.empty((num_item,embed_size))
        self.e = torch.empty((num_item,embed_size))
        self.u = torch.empty((num_item,embed_size))
    
    def setter(self,dynamic_embeddings,static_item_embeddings,user_embedding):
        self.x = dynamic_embeddings
        self.e = static_item_embeddings
        self.u = user_embedding


    def forward(self):
        self.a_u = torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.u))) / torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.u))) + torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.x))) +torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.e)))
        self.a_d = torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.x))) / torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.u))) + torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.x))) +torch.exp(torch.transpose(self.Z_R, 0,0) * torch.tanh(torch.matmul(self.W_R,self.e)))
        return self.a_u , self.a_d

    def calculate_FusionLayer(self):
        u = list(self.u)
        x = list(self.x)
        e = list(self.e)

        # return self.a_u*self.u + self.a_d*self.x + (1 - self.a_d - self.a_u)*self.e
        uau = self.a_u*u
        xad = self.a_d*x
        e1 = (1 - self.a_d - self.a_u)*e

        return torch.Tensor(uau+xad+e1)


class PositionalEncodingLayer(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.n=10000
        self.dropout = nn.Dropout(p=0.5)
        # self.P = torch.zeros((seq_len, embeding_size))

    def calculatePositionEncoding(self,seq_len):
        pe = torch.zeros((seq_len, embed_size))
        for k in range(seq_len):
            for i in np.arange(int(embed_size/2)):
                denominator = np.power(self.n, 2*i/embed_size)
                pe[k, 2*i] = np.sin(k/denominator)
                pe[k, 2*i+1] = np.cos(k/denominator)
        return pe 

    def forward(self, seq_len ,x):
        x = x + self.calculatePositionEncoding(seq_len)
        return self.dropout(x)


class TransformerLayer(nn.Module):
    def __init__(self,d_model, nhead, d_hid, nlayers ):
        super().__init__()
        self.d_model = d_model
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncodingLayer()
        encoder_layers = TransformerEncoderLayer(self.d_model, nhead, d_hid, 0.5)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.decoder = nn.Linear(self.d_model, self.d_model)

    def forward(self,maximum_length,src, src_mask) :
        src = self.pos_encoder(maximum_length,src)+src
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output




# Utils

In [ ]:
import torch

def generate_square_subsequent_mask(sz) :
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

def get_maximum_sequence_length(seq):
    # for masking
    length_of_seqs = []
    for i in seq.values():
        length_of_seqs.append(len(i))

    return max(length_of_seqs)

# Model

In [ ]:
import torch.nn as nn
import torch

from dhg.nn import HGNNConv
from dhg import Hypergraph
from collections import defaultdict

class NeuralSeqRecommender(nn.Module):
    def __init__(self,item_num):
        super().__init__()
        self.item_num = item_num
        self.residual_gating = ResidualGAtingLayer(self.item_num,embed_size)
        self.hconv = HGNNConv(self.item_num,embed_size,True,True,dropout_rate,True)
        self.user_intent = ShortTermUserIntent(self.item_num,embed_size)
        self.fusion = FusionLayer(self.item_num,embed_size)

        d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
        nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
        nhead = 2  # number of heads in nn.MultiheadAttention
        self.transformer = TransformerLayer(embed_size, nhead, d_hid, nlayers)
        
        self.all_dynamic_embeddings = []
        self.intents = {}
        self.interactions = defaultdict(list)

    def set_btch(self,btch):        
        self.batch_num = btch

    def get_first_layer(self , static_item_embeddings , all_hyper_edges):
        static_item_embeddings = torch.Tensor(static_item_embeddings)
        if static_item_embeddings.shape[1] == 300:
            static_item_embeddings = nn.Linear(300,embed_size)(static_item_embeddings)

        # create dynamic embedding
        self.all_dynamic_embeddings = []
        dynamic_embeddings = static_item_embeddings
        # تصحیح ایندکس ها. چون فایلها را قسمت بندی کردیم ولی ایندکس های در ادامه هم هستند . این مشکل اوت آو ایندکس میدهد
        right_index = self.item_num*self.batch_num 

        for h_edge in all_hyper_edges: 
            edg = []
            edg2 = []
            for i in list(h_edge.values()):
                for ii in i: 
                    edg.append(ii-right_index)
                edg2.append(edg)
                edg = []
            # G = Hypergraph(self.item_num,list(h_edge.values())) #number of items(nodes) , list of edges(users)
            G = Hypergraph(self.item_num,edg2)

            dynamic_embeddings = self.hconv(dynamic_embeddings,G)
            dynamic_embeddings = self.hconv(dynamic_embeddings,G)

            self.all_dynamic_embeddings.append(dynamic_embeddings)

            self.residual_gating.setter(dynamic_embeddings,static_item_embeddings)
            dynamic_embeddings = self.residual_gating.calculate_residualGAting()

        conv_dynamic_embeddings = self.all_dynamic_embeddings
        print("first layer is done!")
        return static_item_embeddings , conv_dynamic_embeddings


    def get_second_layer(self,all_hyper_graphs , conv_dynamic_embeddings , dataset_by_year , static_item_embeddings):
        intents = self.create_user_intents(all_hyper_graphs , conv_dynamic_embeddings , dataset_by_year)
        interactions = self.create_interactions(dataset_by_year , intents , conv_dynamic_embeddings , static_item_embeddings)
        
        print("second layer is done!")
        return interactions

    def create_user_intents(self, all_hyper_graphs , conv_dynamic_embeddings , dataset_by_year):
        for graph,embedding,aYear_data,year in zip(all_hyper_graphs,conv_dynamic_embeddings,dataset_by_year.values(),dataset_by_year.keys()):
            userIDs = list(set(aYear_data.reviewerID))
             
            self.user_intent.setter(graph)
            # intent_with_ids = defaultdict(list)
            for u,i in zip(userIDs,self.user_intent(embedding,self.batch_num)):
                # intent_with_ids[u].append(i)
                self.intents[u,year]= i
                # intent_with_ids = defaultdict(list)

        return self.intents

    def create_interactions(self , dataset_by_year , intents , conv_dynamic_embeddings , static_item_embeddings):
        key_list = list(dataset_by_year.keys())
        for year in dataset_by_year.keys():
            df_aYear = dataset_by_year[year]
            for interaction_row in df_aYear.iterrows():
                right_index = self.item_num*self.batch_num 
                
                item_id = interaction_row[1]['asin'] - right_index
                user_id = interaction_row[1]['reviewerID']
                y = key_list.index(year)
                intent_tensor = intents[user_id,year]
                self.fusion.setter(conv_dynamic_embeddings[y][item_id],static_item_embeddings[item_id],intent_tensor)
                self.interactions[user_id,item_id,year].append(self.fusion.calculate_FusionLayer())

        return self.interactions


    def get_third_layer(self , interactions):
        maximum_length = get_maximum_sequence_length(interactions)

        src_mask = generate_square_subsequent_mask(maximum_length)

        outputs = []
        for inters in interactions.values():
            for i in inters:
                output = self.transformer(maximum_length,i, src_mask)
            outputs.append(output)
        print("third layer is done!")
        return outputs

    def get_dynamic_embeddings_by_id(self,year,index):
        return self.all_dynamic_embeddings[year][index]

    def get_user_intent_by_id(self,year,index):
        return self.intents[index,year]

    def get_interactions_by_id(self,index):
        return self.interactions[index]

    def forward(self,static_item_embeddings,all_hyper_edges,all_hyper_graphs,dataset_by_year,btch):
        self.set_btch(btch)
        reshaped_static_embed , conv_dynamic_embeddings = self.get_first_layer(static_item_embeddings , all_hyper_edges)
        interactions = self.get_second_layer(all_hyper_graphs , conv_dynamic_embeddings , dataset_by_year,reshaped_static_embed)
        return reshaped_static_embed , self.get_third_layer(interactions)

# Loss function

In [ ]:
import random
def user_score(user_pref , dynamic_embedding , static_embedding):
    output = dynamic_embedding + torch.Tensor(static_embedding)
    output = torch.matmul(user_pref , output)
    return output


def BPR_loss_function_each_user(data,model,year,user_id,item_id,btch):
    right_index = number_of_item*btch
    item_id = item_id - right_index
    all_years = list(data.get_years())
    item_list = data.get_user_purches_item(user_id)
    u = model.get_user_intent_by_id(year,user_id)
    dynamic_item_embedding = model.get_dynamic_embeddings_by_id(all_years.index(year),item_id)[1]
    static_item_embedding = reshaped_static_embed[item_id]
    y_i = user_score(u , dynamic_item_embedding,static_item_embedding)

    j_items = list(set(list(range(number_of_item))) - set(item_list))
    j = random.choice(j_items)

    dynamic_item_embedding = model.get_dynamic_embeddings_by_id(all_years.index(year),j)[1]
    static_item_embedding = reshaped_static_embed[j]
    y_j = user_score(u , dynamic_item_embedding,static_item_embedding)

    model_parameters = torch.nn.utils.parameters_to_vector(model.parameters())
    model_parameters_2 = torch.pow(model_parameters, 2)
    w = torch.sum(model_parameters_2)

    L2_regularization = lmbd*w

    # return (y_i - y_j) - lmbd 
    return  torch.sigmoid(y_i - y_j) #- L2_regularization 




In [ ]:
import operator
import itertools

def BPR_loss_function(data,model,btch):
    loss_list = {}
    all_user_items = []
    all_user_years = []
    users = data.get_user()
    for user in users:    
        items = data.get_user_purches_item(user)
        years = data.get_user_purches_year(user)
        losses = []
        for item , year in zip(items,years):
            losses.append(int(BPR_loss_function_each_user(data,model,year,user,item,btch)))
        loss_list[user]=sum(losses)
    
    return loss_list

# Calculating Ru

In [ ]:
from collections import defaultdict
from operator import itemgetter
import random

def calculating_Ru(data , model , reshaped_static_embed , btch):

    rank_of_test_data = defaultdict(list)
    ranked_items_negative = []
    leave_one_out_list = []
    all_user_items = []
    all_user_years = []

    all_years = list(data.get_years())

    for i in data.get_user():    
        all_user_items.extend(data.get_user_purches_item(i))
        all_user_years.extend(data.get_user_purches_year(i))    
        
        for j , k in zip(data.get_user_purches_item(i),data.get_user_purches_year(i)):

            

            test_index = all_user_items.index(j)
            
            test_item = all_user_items[test_index] 
            test_year = all_user_years[test_index]

            all_user_items.remove(j)
            all_user_years.remove(k)

            #be khater namgozari behtar
            positive_years = all_user_years
            positive_items = all_user_items

            
            if len(positive_items) > 0 :
                range_of_number = [item for item in range(number_of_item*btch, number_of_item*(btch+1))]
                pItem = list(set(data.get_user_purches_item(i)))
                for l in pItem:
                    range_of_number.remove(l)
                # negative_items = list(set(list(range(number_of_item))) - set(data.get_user_purches_item(i)))
                negative_items = random.sample(range_of_number , len(positive_items))
                negative_items.append(test_item)
                negative_years = positive_years + [test_year]
            else : #never happened, just for testing
                negative_items = test_item
                negative_years = test_year

            for item,year in zip(negative_items,negative_years): 
                item = item - (number_of_item*btch)
                u = model.get_user_intent_by_id(year,i)
                dynamic_item_embedding = model.get_dynamic_embeddings_by_id(all_years.index(year),item)
                static_item_embedding = reshaped_static_embed[item]
                score = int(user_score(u , dynamic_item_embedding,static_item_embedding)) 
                item_score = (item,score)
                ranked_items_negative.append(item_score)

            ranked_items_negative = sorted(ranked_items_negative,key=itemgetter(1) , reverse=True)
            r_u = 0
            r_u_item = 0
            test_item = test_item - number_of_item*btch
            for x, y in enumerate(ranked_items_negative):
                if y[0] == test_item :
                    r_u = y[1] 
                    r_u_item = y[0] 
            leave_one_out_list.append(ranked_items_negative.index((r_u_item,r_u))+1)

            all_user_items = []
            all_user_years = []
            ranked_items_negative = []

            all_user_items.extend(data.get_user_purches_item(i))
            all_user_years.extend(data.get_user_purches_year(i))

        rank_of_test_data[i].append(leave_one_out_list)
        leave_one_out_list = []
        all_user_items = []
        all_user_years = []
        
    for i,j in rank_of_test_data.items():
        rank_of_test_data[i] = np.mean(j) 
        
    return rank_of_test_data 

    

# Evaluation

In [ ]:
import math

def hit_rate(r_u , k):
    if r_u <= k:
        return 1
    else:
        return 0

def NDGC(r_u , k):
    if r_u <= k:
        return 1/math.log((r_u+1), 2)
    else :
        return 0

In [ ]:
def eval_functions(rank_of_test_data):
    hit_rate_list = defaultdict(list)
    for i,j in rank_of_test_data.items():
        hit_rate_list[i].append(hit_rate(j,K))

    NDGC_list = defaultdict(list)
    for i,j in rank_of_test_data.items():
        NDGC_list[i].append(NDGC(j,K))

    r_u_list = list(rank_of_test_data.values())
    r_u_list_reverse = []
    for i in r_u_list:
        if i == 0:
            r_u_list_reverse.append(0)
        else:
            r_u_list_reverse.append(1/i)
    
    return np.mean(r_u_list_reverse) , np.mean(list(NDGC_list.values())) , np.mean(list(hit_rate_list.values()))


# Traning Loop

In [ ]:
data = Data()
model = NeuralSeqRecommender(number_of_item)
optimizer = torch.optim.Adam(model.parameters(),lr=0.005,betas=(0.9,0.999),eps=1e-08,weight_decay=0,amsgrad=False)

In [ ]:
import os 
from collections import Counter
for e in range(epochs):
    print('\n')
    print("epoch {} starts :".format(e) )
    list_of_loss = {}
    list_of_eval = {}
    common_keys_loss = []
    common_keys_eval = []
    for i in range(0,340):
        data.clear_every_thing()
        data.set_data('features/feat_{}.csv'.format(i),'interactions/inter_{}.csv'.format(i))
        data.set_static_embedding()
        number_of_item = data.get_statistic()[0]
        number_of_user = data.get_statistic()[1]
        number_of_interactions = data.get_statistic()[2]

        print('epoch number {} , feature file number {} : number of items: '.format(e,i),number_of_item)
        print('epoch number {} , feature file number {}" : number of user: '.format(e,i),number_of_user)
        print('epoch number {} , interaction file number {}" : number of interactions: '.format(e,i),number_of_interactions)

        optimizer.zero_grad()
        # model = NeuralSeqRecommender(number_of_item)
        reshaped_static_embed , output = model(data.static_item_embedding , data.get_all_hyper_edges(),data.get_all_hyper_graphs() , data.get_dataset_by_year(),i)

        #loss
        if i==0 :
            list_of_loss= BPR_loss_function(data,model,i)
        else:
            tempListLoss = BPR_loss_function(data,model,i)
            all_loss = dict(Counter(list_of_loss) + Counter(tempListLoss))
            common_keys_loss.append(set(all_loss).intersection(tempListLoss))
            
            list_of_loss = {}
            list_of_loss = all_loss.copy()
            all_loss = {} 
        
            
        

        #evaluations
        if i==0 :
            list_of_eval= calculating_Ru(data , model , reshaped_static_embed,i)
        else:
            tempList = calculating_Ru(data , model , reshaped_static_embed,i)
            all_evaluation = dict(Counter(list_of_eval) + Counter(tempList))
            common_keys_eval.append(set(all_evaluation).intersection(tempList))
            
            list_of_eval = {}
            list_of_eval = all_evaluation.copy()
            all_evaluation = {}


        optimizer.step()

   # loss
    lcommon_keys = []
    for i in common_keys_loss:
        for j in i:
            lcommon_keys.append(j)
    lcommon_keys = list(set(lcommon_keys))
    for k in lcommon_keys:
        list_of_loss[k]=list_of_loss[k]/5

    u = 0
    for ii in list_of_loss.values():
        u +=ii
    u = u/len(list_of_loss)
    print('epoch {} : loss = '.format(e)  , u)

    #evaluation
    ecommon_keys = []
    for i in common_keys_eval:
        for j in i:
            ecommon_keys.append(j)
    ecommon_keys = list(set(ecommon_keys))
    for k in ecommon_keys:
        list_of_eval[k]=list_of_eval[k]/5
    evaluation_result = eval_functions(list_of_eval)


    print('epoch {} : MRR'.format(e) ,evaluation_result[0])
    print('epoch {} : NDGC@3'.format(e) ,evaluation_result[1])
    print('epoch {} : hit_rate@3'.format(e) ,evaluation_result[2])
    